In [6]:
!pip install pulp

In [7]:
import pandas as pd
from pulp.apis import COIN_CMD

# Load bid data
bid_data = pd.read_csv("./HW1_BidData.csv", index_col=0)

students = bid_data.index
internships = bid_data.columns

#Problem1
model = LpProblem("Internship_Assignment", LpMaximize)

# Decision variables
x = {(i, j): LpVariable(f"x_{i}_{j}", cat="Binary") for i in students for j in internships}

# Objective: Maximize total bid points
model += lpSum(bid_data.loc[i, j] * x[i, j] for i in students for j in internships)

# Constraints:
# Each student is assigned exactly 2 internships
for i in students:
    model += lpSum(x[i, j] for j in internships) == 2

# Each internship can take at most 5 students
for j in internships:
    model += lpSum(x[i, j] for i in students) <= 5

# Solve model
model.solve(COIN_CMD())

# Print results
assignments = [(i, j) for i in students for j in internships if x[i, j].value() == 1]
assignments_df = pd.DataFrame(assignments, columns=["Student", "Internship"])
print(assignments_df)


Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - cbc /var/folders/fr/s7hf4z3910lfdqrs68zly6640000gn/T/9f0517675d5842d987de6182fc35ebf1-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/fr/s7hf4z3910lfdqrs68zly6640000gn/T/9f0517675d5842d987de6182fc35ebf1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 507 RHS
At line 530 BOUNDS
At line 636 ENDATA
Problem MODEL has 22 rows, 105 columns and 210 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1160 - 0.00 seconds
Cgl0004I processed model has 22 rows, 105 columns (105 integer (105 of which binary)) and 210 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 1160
Cbc0038I Before mini branch and bound, 105 integers at bound fixed and 0 continuous
Cbc0038I M

In [9]:
from pulp import LpMinimize, LpMaximize, LpProblem, LpVariable, lpSum

# Define model
model = LpProblem("Nut_Company_Production", LpMaximize)

# Decision variables (pounds of each product)
x_w = LpVariable("Whole", lowBound=1000)  # At least 1000 lbs
x_c = LpVariable("Cluster", lowBound=400, upBound=500)
x_r = LpVariable("Crunch", lowBound=0, upBound=150)
x_k = LpVariable("Roasted", lowBound=0, upBound=200)

# Objective function: Maximize total VM
model += 1.93 * x_w + 1.04 * x_c + 1.15 * x_r + 1.33 * x_k

# Constraints:
model += 0.6 * x_w + 0.4 * x_c + 0.2 * x_r + 1.0 * x_k <= 1100  # Macadamia nuts
model += 0.4 * x_w + 0.6 * x_c + 0.8 * x_r <= 800  # Chocolate

# Machine constraints (60 hours = 3600 minutes)
model += x_w + x_c + x_r + x_k <= 3600  # Hulling
model += 2 * x_w + 1.5 * x_c + 1 * x_r + 4 * x_k <= 3600  # Roasting
model += x_w + 0.7 * x_c + 0.2 * x_r <= 3600  # Coating
model += 2.5 * x_w + 1.6 * x_c + 1.25 * x_r + x_k <= 3600  # Packaging

# Solve model
model.solve(COIN_CMD())

# Print results
print(f"Whole: {x_w.value()} lbs")
print(f"Cluster: {x_c.value()} lbs")
print(f"Crunch: {x_r.value()} lbs")
print(f"Roasted: {x_k.value()} lbs")
print(f"Total Variable Margin: ${model.objective.value():.2f}")


Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - cbc /var/folders/fr/s7hf4z3910lfdqrs68zly6640000gn/T/afa8b904641244498dd5249db1b1a638-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/fr/s7hf4z3910lfdqrs68zly6640000gn/T/afa8b904641244498dd5249db1b1a638-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 38 RHS
At line 45 BOUNDS
At line 51 ENDATA
Problem MODEL has 6 rows, 4 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-2) rows, 4 (0) columns and 15 (-7) elements
0  Obj 2346 Dual inf 5.7966627 (4)
2  Obj 2839.075
Optimal - objective value 2839.075
After Postsolve, objective 2839.075, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2839.075 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallc

In [10]:
# Define lease durations and costs
lease_costs = {1: 50, 2: 90, 3: 135, 4: 160, 5: 200}  # Cost per sq ft
months = [1, 2, 3, 4, 5]
space_required = {1: 30000, 2: 25000, 3: 40000, 4: 20000, 5: 10000}

# Define model
model = LpProblem("Movie_Storage_Optimization", LpMinimize)

# Decision variables
x = {(t, d): LpVariable(f"x_{t}_{d}", lowBound=0) for t in months for d in range(1, 6)}

# Objective function: Minimize total cost
model += lpSum(lease_costs[d] * x[t, d] for t in months for d in range(1, 6))

# Constraints: Ensure enough storage in each month
for m in months:
    model += lpSum(x[t, d] for t in range(1, m + 1) for d in range(m - t + 1, 6)) >= space_required[m]

# Solve model
model.solve(COIN_CMD())

# Print results
lease_plan = [(t, d, x[t, d].value()) for t in months for d in range(1, 6) if x[t, d].value() > 0]
lease_plan_df = pd.DataFrame(lease_plan, columns=["Start Month", "Duration", "Sq Ft Leased"])
print(lease_plan_df)


Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - cbc /var/folders/fr/s7hf4z3910lfdqrs68zly6640000gn/T/0b710e052b024cd3954a012978ee1b8e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/fr/s7hf4z3910lfdqrs68zly6640000gn/T/0b710e052b024cd3954a012978ee1b8e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 91 RHS
At line 97 BOUNDS
At line 98 ENDATA
Problem MODEL has 5 rows, 25 columns and 55 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 15 (-10) columns and 35 (-20) elements
0  Obj 0 Primal inf 125000 (5)
5  Obj 5275000
Optimal - objective value 5275000
After Postsolve, objective 5275000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 5275000 - 5 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seco